In [1]:
import os

In [2]:
os.chdir("../")                           

In [4]:
%pwd

'c:\\Users\\yashr\\OneDrive\\Desktop\\Text_Summarization_project\\Text-summarizer-app'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path



In [ ]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_directories



In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

In [ ]:
import os 
import urllib.request as request
import zipfile

from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size



In [ ]:
class DataIngestion:
    def __init__(self) -> None:
        pass